# Lesson 2: RAG Triad of metrics

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import utils

import os
import openai
openai.api_key = utils.get_openai_api_key()

In [3]:
from trulens_eval import Tru

tru = Tru()
tru.reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


In [4]:
from llama_index import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["./eBook-How-to-Build-a-Career-in-AI.pdf"]
).load_data()

In [5]:
from llama_index import Document

document = Document(text="\n\n".\
                    join([doc.text for doc in documents]))

In [6]:
from utils import build_sentence_window_index

from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

sentence_index = build_sentence_window_index(
    document,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="sentence_index"
)

In [7]:
from utils import get_sentence_window_query_engine

sentence_window_engine = \
get_sentence_window_query_engine(sentence_index)

In [8]:
output = sentence_window_engine.query(
    "How do you create your AI portfolio?")
output.response

'To create your AI portfolio, you should focus on building a collection of projects that demonstrate your skill progression. This can be achieved by starting with simpler projects and gradually increasing the complexity as you gain more experience. Additionally, it is important to ensure that your portfolio showcases a variety of AI techniques and applications to highlight your versatility and breadth of knowledge. By following this approach, you can effectively showcase your skills and attract potential employers in the field of AI.'

## Feedback functions

In [9]:
import nest_asyncio

nest_asyncio.apply()

In [10]:
from trulens_eval import OpenAI as fOpenAI

provider = fOpenAI(api_key="Your api_key")


### 1. Answer Relevance

In [11]:
from trulens_eval import Feedback

f_qa_relevance = Feedback(
    provider.relevance_with_cot_reasons,
    name="Answer Relevance"
).on_input_output()

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .


### 2. Context Relevance

In [12]:
from trulens_eval import TruLlama

context_selection = TruLlama.select_source_nodes().node.text

In [13]:
import numpy as np

f_qs_relevance = (
    Feedback(provider.qs_relevance,
             name="Context Relevance")
    .on_input()
    .on(context_selection)
    .aggregate(np.mean)
)

✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input statement will be set to __record__.app.query.rets.source_nodes[:].node.text .


In [14]:
import numpy as np

f_qs_relevance = (
    Feedback(provider.qs_relevance_with_cot_reasons,
             name="Context Relevance")
    .on_input()
    .on(context_selection)
    .aggregate(np.mean)
)

✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input statement will be set to __record__.app.query.rets.source_nodes[:].node.text .


### 3. Groundedness

In [15]:
from trulens_eval.feedback import Groundedness

grounded = Groundedness(groundedness_provider=provider)

In [16]:
f_groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons,
             name="Groundedness"
            )
    .on(context_selection)
    .on_output()
    .aggregate(grounded.grounded_statements_aggregator)
)

✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


## Evaluation of the RAG application

In [17]:
from trulens_eval import TruLlama
from trulens_eval import FeedbackMode

tru_recorder = TruLlama(
    sentence_window_engine,
    app_id="App_1",
    feedbacks=[
        f_qa_relevance,
        f_qs_relevance,
        f_groundedness
    ]
)

In [18]:
eval_questions = []
with open('eval_questions.txt', 'r') as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        eval_questions.append(item)

In [19]:
eval_questions

['What are the keys to building a career in AI? 한글로 답변해줘',
 'How can teamwork contribute to success in AI? 한글로 답변해줘',
 'What is the importance of networking in AI? 한글로 답변해줘',
 'What are some good habits to develop for a successful career? 한글로 답변해줘',
 'How can altruism be beneficial in building a career? 한글로 답변해줘',
 'What is imposter syndrome and how does it relate to AI? 한글로 답변해줘',
 'Who are some accomplished individuals who have experienced imposter syndrome? 한글로 답변해줘',
 'What is the first step to becoming good at AI? 한글로 답변해줘',
 'What are some common challenges in AI? 한글로 답변해줘',
 'Is it normal to find parts of AI challenging? 한글로 답변해줘']

In [20]:
eval_questions.append("How can I be successful in AI?")

In [21]:
eval_questions

['What are the keys to building a career in AI? 한글로 답변해줘',
 'How can teamwork contribute to success in AI? 한글로 답변해줘',
 'What is the importance of networking in AI? 한글로 답변해줘',
 'What are some good habits to develop for a successful career? 한글로 답변해줘',
 'How can altruism be beneficial in building a career? 한글로 답변해줘',
 'What is imposter syndrome and how does it relate to AI? 한글로 답변해줘',
 'Who are some accomplished individuals who have experienced imposter syndrome? 한글로 답변해줘',
 'What is the first step to becoming good at AI? 한글로 답변해줘',
 'What are some common challenges in AI? 한글로 답변해줘',
 'Is it normal to find parts of AI challenging? 한글로 답변해줘',
 'How can I be successful in AI?']

In [22]:
for question in eval_questions:
    with tru_recorder as recording:
        sentence_window_engine.query(question)

Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICallback is registered for handling create but there are no endpoints waiting to receive the result.
Callback class OpenAICal

In [23]:
records, feedback = tru.get_records_and_feedback(app_ids=[])
records.head()

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,Answer Relevance,Groundedness,Context Relevance,Answer Relevance_calls,Groundedness_calls,Context Relevance_calls,latency,total_tokens,total_cost
0,App_1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_e8753ab1316abeba09af6b3c400b286a,"""What are the keys to building a career in AI?...","""AI \uacbd\ub825\uc744 \uc313\ub294 \ub370 \ud...",-,"{""record_id"": ""record_hash_e8753ab1316abeba09a...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2024-01-24T18:50:20.222231"", ""...",2024-01-24T18:50:29.031457,1.0,1.0,0.4,[{'args': {'prompt': 'What are the keys to bui...,[{'args': {'source': 'Many companies are stil...,[{'args': {'question': 'What are the keys to b...,8,0,0.0
1,App_1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_effdeac185ec58ed677c6052c491a1f4,"""How can teamwork contribute to success in AI?...","""\ud300\uc6cc\ud06c\ub294 AI\uc5d0\uc11c \uc13...",-,"{""record_id"": ""record_hash_effdeac185ec58ed677...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2024-01-24T18:50:29.374421"", ""...",2024-01-24T18:50:43.384402,0.8,1.0,0.4,[{'args': {'prompt': 'How can teamwork contrib...,[{'args': {'source': 'To get a project starte...,[{'args': {'question': 'How can teamwork contr...,14,0,0.0
2,App_1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_9fb8268af956b1ec3c29068fc7cd7fa3,"""What is the importance of networking in AI? \...","""AI\uc5d0\uc11c \ub124\ud2b8\uc6cc\ud0b9\uc758...",-,"{""record_id"": ""record_hash_9fb8268af956b1ec3c2...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2024-01-24T18:50:43.714030"", ""...",2024-01-24T18:50:54.921048,0.8,0.6,0.3,[{'args': {'prompt': 'What is the importance o...,[{'args': {'source': 'Such conversations are s...,[{'args': {'question': 'What is the importance...,11,0,0.0
3,App_1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_b591136f398c2bb6aae158e82b266985,"""What are some good habits to develop for a su...","""\uc131\uacf5\uc801\uc778 \uacbd\ub825\uc744 \...",-,"{""record_id"": ""record_hash_b591136f398c2bb6aae...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2024-01-24T18:50:55.253760"", ""...",2024-01-24T18:51:03.493554,1.0,1.0,0.5,[{'args': {'prompt': 'What are some good habit...,[{'args': {'source': 'Although it may seem lik...,[{'args': {'question': 'What are some good hab...,8,0,0.0
4,App_1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_84242d37ac888532558c42f217bba0f1,"""How can altruism be beneficial in building a ...","""\ub2e4\ub978 \uc0ac\ub78c\ub4e4\uc744 \ub3d5\...",-,"{""record_id"": ""record_hash_84242d37ac888532558...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2024-01-24T18:51:03.816334"", ""...",2024-01-24T18:51:22.575087,0.9,0.2,0.6,[{'args': {'prompt': 'How can altruism be bene...,[{'args': {'source': 'Many successful people d...,[{'args': {'question': 'How can altruism be be...,18,0,0.0


In [24]:
import pandas as pd

pd.set_option("display.max_colwidth", None)
records[["input", "output"] + feedback]

,input,output,Groundedness,Context Relevance,Answer Relevance
0,"""What are the keys to building a career in AI? \ud55c\uae00\ub85c \ub2f5\ubcc0\ud574\uc918""","""AI \uacbd\ub825\uc744 \uc313\ub294 \ub370 \ud544\uc694\ud55c \ud575\uc2ec \uc694\uc18c\ub294 \ub2e4\uc74c\uacfc \uac19\uc2b5\ub2c8\ub2e4: \ud575\uc2ec \uae30\uc220\uc744 \ubc30\uc6b0\ub294 \uac83, \ud504\ub85c\uc81d\ud2b8\uc5d0 \ucc38\uc5ec\ud558\ub294 \uac83, \uc801\ud569\ud55c \uc77c\uc790\ub9ac\ub97c \ucc3e\ub294 \uac83, \uadf8\ub9ac\uace0 \uc9c0\uc6d0 \ucee4\ubba4\ub2c8\ud2f0\uc5d0 \uc18d\ud558\ub294 \uac83\uc785\ub2c8\ub2e4. \uc774\ub7ec\ud55c \ub2e8\uacc4\ub97c \ub530\ub77c\uac00\uba74\uc11c AI \uacbd\ub825\uc744 \uad6c\ucd95\ud560 \uc218 \uc788\uc2b5\ub2c8\ub2e4.""",1.000000,0.4,1.0
1,"""How can teamwork contribute to success in AI? \ud55c\uae00\ub85c \ub2f5\ubcc0\ud574\uc918""","""\ud300\uc6cc\ud06c\ub294 AI\uc5d0\uc11c \uc131\uacf5\uc744 \uc774\ub8e8\ub294 \ub370 \uae30\uc5ec\ud560 \uc218 \uc788\uc2b5\ub2c8\ub2e4. \ub300\ud615 AI \ud504\ub85c\uc81d\ud2b8\ub97c \uc218\ud589\ud560 \ub54c \ud300\uc6cc\ud06c \ub2a5\ub825\uc740 \ub354\uc6b1 \uc911\uc694\ud574\uc9c0\uba70, \ud615\uc2dd\uc801\uc778 \ub9ac\ub354\uc2ed \uc9c1\ucc45\uc5d0 \uc788\ub4e0 \uc5c6\ub4e0 \ud504\ub85c\uc81d\ud2b8\ub97c \uc774\ub04c\uc5b4 \ub098\uac08 \uc218 \uc788\ub294 \ub2a5\ub825\uc774 \ud544\uc694\ud569\ub2c8\ub2e4. \uae30\uc220\uc801\uc778 \ud1b5\ucc30\ub825\uc744 \ubc14\ud0d5\uc73c\ub85c \ud504\ub85c\uc81d\ud2b8\ub97c \uc774\ub04c\uc5b4 \ub098\uac00\ub294 \ub2a5\ub825\uc740 \ub9ac\ub354\ub85c\uc11c \uc131\uc7a5\ud558\ub294 \ub370 \ub3c4\uc6c0\uc774 \ub418\uba70, \ud504\ub85c\uc81d\ud2b8\uc758 \ud488\uc9c8\uc744 \ud06c\uac8c \ud5a5\uc0c1\uc2dc\ud0ac \uc218 \uc788\uc2b5\ub2c8\ub2e4.""",1.000000,0.4,0.8
2,"""What is the importance of networking in AI? \ud55c\uae00\ub85c \ub2f5\ubcc0\ud574\uc918""","""AI\uc5d0\uc11c \ub124\ud2b8\uc6cc\ud0b9\uc758 \uc911\uc694\uc131\uc740 \ucde8\uc5c5\uc744 \uc704\ud574 \ud544\uc694\ud55c \uc815\ubcf4\ub97c \uc5bb\uace0, \ub2e4\uc591\ud55c \uc0ac\ub78c\ub4e4\uacfc \uc5f0\uacb0\ub418\uc5b4 \uae30\ud68c\ub97c \ucc3e\uc744 \uc218 \uc788\ub294 \uac83\uc785\ub2c8\ub2e4. \ub124\ud2b8\uc6cc\ud0b9\uc744 \ud1b5\ud574 \ub2e4\ub978 \uc0ac\ub78c\ub4e4\uc758 \uacbd\ud5d8\uacfc \ud1b5\ucc30\ub825\uc744 \uc5bb\uc744 \uc218 \uc788\uc73c\uba70, \uad00\ub828 \ubd84\uc57c\uc5d0\uc11c \uc77c\ud558\ub294 \uc0ac\ub78c\ub4e4\uacfc\uc758 \uad00\uacc4\ub97c \uad6c\ucd95\ud560 \uc218 \uc788\uc2b5\ub2c8\ub2e4. \uc774\ub294 \ucde8\uc5c5 \uc900\ube44\uc640 \uc9c4\ub85c \uac1c\ubc1c\uc5d0 \ub3c4\uc6c0\uc774 \ub420 \uc218 \uc788\uc2b5\ub2c8\ub2e4.""",0.600000,0.3,0.8
3,"""What are some good habits to develop for a successful career? \ud55c\uae00\ub85c \ub2f5\ubcc0\ud574\uc918""","""\uc131\uacf5\uc801\uc778 \uacbd\ub825\uc744 \uc704\ud574 \uac1c\ubc1c\ud560 \uc218 \uc788\ub294 \uba87 \uac00\uc9c0 \uc88b\uc740 \uc2b5\uad00\uc740 \uc2dd\uc0ac, \uc6b4\ub3d9, \uc218\uba74, \uac1c\uc778\uc801\uc778 \uad00\uacc4, \uc5c5\ubb34, \ud559\uc2b5 \ubc0f \uc790\uae30 \uad00\ub9ac\uc5d0 \ub300\ud55c \uac83\uc785\ub2c8\ub2e4. \uc774\ub7ec\ud55c \uc2b5\uad00\ub4e4\uc740 \uac74\uac15\uc744 \uc720\uc9c0\ud558\uba74\uc11c \uc804\uc9c4\ud558\ub294 \ub370 \ub3c4\uc6c0\uc774 \ub429\ub2c8\ub2e4.""",1.000000,0.5,1.0
4,"""How can altruism be beneficial in building a career? \ud55c\uae00\ub85c \ub2f5\ubcc0\ud574\uc918""","""\ub2e4\ub978 \uc0ac\ub78c\ub4e4\uc744 \ub3d5\ub294 \uac83\uc740 \uc6b0\ub9ac \uc790\uc2e0\uc758 \uc131\uacf5\uc5d0 \ub3c4\uc6c0\uc774 \ub420 \uc218 \uc788\uc2b5\ub2c8\ub2e4. \ub2e4\ub978 \uc0ac\ub78c\ub4e4\uc744 \ub3d5\ub294 \uac83\uc740 \uc6b0\ub9ac\uc758 \uce5c\ubc00\ud55c \uad00\uacc4\ub97c \ud615\uc131\ud558\uace0, \uc6b0\ub9ac\uc758 \uc2e0\ub8b0\ub97c \uc5bb\uc744 \uc218 \uc788\uc73c\uba70, \uc6b0\ub9ac\uc758 \ub124\ud2b8\uc6cc\ud06c\ub97c \ud655\uc7a5\ud558\ub294 \ub370 \ub3c4\uc6c0\uc774 \ub420 \uc218 \uc788\uc2b5\ub2c8\ub2e4. \uc774\ub7ec\ud55c \uad00\uacc4\uc640 \ub124\ud2b8\uc6cc\ud06c\

In [27]:
tru.get_leaderboard(app_ids=[])

,Groundedness,Context Relevance,Answer Relevance,latency,total_cost
app_id,,,,,
App_1,0.839394,0.554545,0.881818,12.545455,0.0


In [28]:
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path:   Network URL: http://10.10.1.8:8501



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>